In [1]:
import numpy as np
#from numba import njit
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [2]:
a = np.linspace(0,49,50).reshape(5,10)
a

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.]])

In [3]:
np.delete(a, [3], axis=1)

array([[ 0.,  1.,  2.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 44., 45., 46., 47., 48., 49.]])

In [4]:
np.average(a, axis=0)

array([20., 21., 22., 23., 24., 25., 26., 27., 28., 29.])

In [5]:
b = np.linspace(0,4,5)
b[None,:]

array([[0., 1., 2., 3., 4.]])

In [7]:
b[:0]

array([], dtype=float64)

In [31]:
np.cumsum(b)

array([ 0.,  1.,  3.,  6., 10.])

In [30]:
a*b[:,None]

ValueError: operands could not be broadcast together with shapes (5,10) (1,5) 

In [14]:
c = np.linspace(25,49,25).reshape((5,5))
c

array([[25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34.],
       [35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44.],
       [45., 46., 47., 48., 49.]])

In [15]:
a*c

array([[   0.,   26.,   54.,   84.,  116.],
       [ 150.,  186.,  224.,  264.,  306.],
       [ 350.,  396.,  444.,  494.,  546.],
       [ 600.,  656.,  714.,  774.,  836.],
       [ 900.,  966., 1034., 1104., 1176.]])

In [9]:
from chainconsumer import ChainConsumer

parameter_names = ['Crab K', 'Crab index', '1A K', '1A index', '4U05 K', '4U05 index', '4U06 K', '4U06 index', 'geminga K', 'geminga index',"z"]
c = ChainConsumer()
chain = np.loadtxt('./chains/1-post_equal_weights.dat')
c.add_chain(chain, parameters=parameter_names, name='fit')

summary = c.analysis.get_summary(parameters=parameter_names)

In [17]:
f"{summary['geminga index'][0]:.5}"

TypeError: unsupported format string passed to NoneType.__format__

In [16]:
for key, value in summary.items():
    print(key,value)

Crab K [0.004556877936842222, 0.0045830281837448954, 0.004621092276687108]
Crab index [-2.0118206813552115, -1.9928552844952536, -1.9705499767840853]
1A K [0.007214110179877939, 0.0072597048693060105, 0.00729193913511652]
1A index [-2.758841113967041, -2.7496725432511413, -2.7303540857511126]
4U05 K [5.5088085969703795e-06, 6.745472106374519e-06, 8.97819304490124e-06]
4U05 index [-9.992452410462825, -9.992452410462825, -9.52685784510881]
4U06 K [3.1339597202339715e-08, 3.1339597202339715e-08, 2.1890734033446788e-07]
4U06 index [7.212429289827883, 9.728599878939361, 9.979305281839961]
geminga K [3.7215659587056573e-08, 3.7215659587056573e-08, 2.5946801963079533e-07]
geminga index [None, 9.044799954824072, None]
z [50345906.30480682, 50345908.66783022, 50345910.170552894]
